In [1]:
# Import the requests library.
import requests
import numpy as np
import pandas as pd
# Import the requests library.
import requests
import datetime
from datetime import datetime
import matplotlib.pyplot as plt
# Dependencies for the wine API
import urllib
import json
# Import the API key.
from config import weather_api_key
import calendar

Token = 'tylRGHbxqiZUDNymMowCBFCuEqeULiWk'
API_Token='Token d8f94b93417333267ca0edcca7dc14a9e035e1bb'

In [2]:
# Import the API key.
headers = {
    'accept': 'application/json',
    'authorization': API_Token
    }

# Red Wine in USA

In [3]:
req_red_USA = urllib.request.Request('https://api.globalwinescore.com/globalwinescores/latest/?color=red&limit=5000&country=Usa', data=None, headers=headers, origin_req_host=None, unverifiable=False, method=None)
res_red_USA = urllib.request.urlopen(req_red_USA).read().decode()
obj_red_USA = json.loads(res_red_USA)
#read json into DF
red_USA_df = pd.DataFrame.from_dict(obj_red_USA['results'], orient='columns')
# To delete the column without having to reassign df you can do:
red_USA_df.drop([ 'wine_slug', 'appellation_slug',  'date',  'lwin', 'lwin_11'],axis=1, inplace=True)
#'wine_id','wine_type','classification',
#Drop NA values
# Red_USA_df = Red_USA_df.dropna()
#convert vintage to date type
red_USA_df['vintage']= pd.to_datetime(red_USA_df['vintage']).dt.year
red_USA_df['regions']=red_USA_df['regions'].str.get(0)
red_USA_df.to_csv("Red_Wine_USA.csv")
red_USA_df

,wine,wine_id,appellation,color,wine_type,regions,country,classification,vintage,is_primeurs,score,confidence_index,journalist_count
0,"Ridge Vineyards, Monte Bello, Santa Cruz Mount...",120786,Santa Cruz Mountains,Red,,California,Usa,None,2013,False,95.83,C+,8
1,"Quintessa, Rutherford",118360,Rutherford,Red,,California,Usa,None,2015,False,95.77,C,3
2,"Harlan Estate, Napa Valley",81980,Napa Valley,Red,,California,Usa,None,2001,False,98.89,A+,6
3,"Dalla Valle Vineyards, Maya, Napa Valley",48290,Napa Valley,Red,,California,Usa,None,2001,False,96.49,B+,5
4,"Philip Togni Vineyard, Cabernet Sauvignon, Nap...",115154,Napa Valley,Red,,California,Usa,None,2001,False,96.20,B,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4359,"Pine Ridge Vineyards, Rutherford Cabernet Sauv...",116343,Napa Valley,Red,,California,Usa,None,1995,False,85.97,C+,3
4360,"La Jota Vineyard, Cabernet Sauvignon, Howell M...",92780,Howell Mountain,Red,,California,Usa,None,1995,False,89.51,C,3
4361,"Pine Ridge Vineyards, Stags Leap District Cabe...",116344,Napa Valley,Red,,California,Usa,None,1995,False,84.59,C,3
4362,"Flora Springs, Sangiovese, Napa Valley",74746,Napa Valley,Red,,California,Usa,None,1996,False,85.51,C,3


# White Wine in USA

In [4]:
req_white_USA = urllib.request.Request('https://api.globalwinescore.com/globalwinescores/latest/?color=white&limit=5000&country=Usa', data=None, headers=headers, origin_req_host=None, unverifiable=False, method=None)
res_white_USA = urllib.request.urlopen(req_white_USA).read().decode()
obj_white_USA = json.loads(res_white_USA)
#read json into DF
white_USA_df = pd.DataFrame.from_dict(obj_white_USA['results'], orient='columns')
# To delete the column without having to reassign df you can do:
white_USA_df.drop([ 'wine_slug', 'appellation_slug',  'date',  'lwin', 'lwin_11'],axis=1, inplace=True)
#'wine_id','wine_type','classification',
#Drop NA values
# white_USA_df = white_USA_df.dropna()
#convert vintage to date type
white_USA_df['vintage']= pd.to_datetime(white_USA_df['vintage']).dt.year
white_USA_df['regions']=white_USA_df['regions'].str.get(0)
white_USA_df.to_csv("White_Wine_USA.csv")

# Weather

# California Avg Temp and Precipitation

In [5]:
#Create average temperature
dates_temp = []
temps = []
# zip_codes=['USW00023129','USC00046074', 'USW00023129']
zip_codes=['USW00023129']
# zip_codes=['USW00023129']


#for each year from 2015-2019 ...
for year in range(2013, 2014):
    year = str(year)
    print('working on year '+year)
    
    #make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00023129&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are average temperature readings
    avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
    #get the date field from all average temperature readings
    dates_temp += [item['date'] for item in avg_temps]
    #get the actual average temperature from all average temperature readings
    temps += [item['value'] for item in avg_temps]

#initialize dataframe
df_temp = pd.DataFrame()

#populate date and average temperature fields (cast string date to datetime
df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
#convert to degrees F
df_temp['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps]
df_temp = df_temp.set_index('date').resample('M').mean()
df_temp

working on year 2013


,avgTemp
date,
2013-04-30,62.678000
2013-05-31,67.123226
2013-06-30,68.300000
2013-07-31,70.316774
2013-08-31,70.432903
2013-09-30,72.326000
2013-10-31,65.950323
2013-11-30,62.612000
2013-12-31,57.072258


In [6]:
# requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:{zip_}&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})

NameError: name 'zip_' is not defined

In [ ]:
### initialize lists to store data
dates_prcp = []
prcp = []
zip_codes=['USW00023129']
#for each year from 2015-2019 ...
for i in range(len(zip_codes)):
    zip_=zip_codes[i]
    for year in range(1992, 2017):
        year = str(year)
    #     print('working on year '+year)

        #make the api call
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&locations?locationid=ZIP:{zip_}&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
        #load the api response as a json
        d = json.loads(r.text)
        #get all items in the response which are Precipitation temperature readings
        avg_prcp = [item for item in d['results'] if item['datatype']=='PRCP']
        #get the date field from all average Precipitation readings
        dates_prcp += [item['date'] for item in avg_prcp]
        #get the actual average Precipitation from all average Precipitation readings
        prcp += [item['value'] for item in avg_prcp]

#initialize dataframe
df_prcp = pd.DataFrame()

#populate date and average Precipitation fields (cast string date to datetime
df_prcp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp]
#convert to mm to inches
df_prcp['avgPrcp'] = [float(v)*0.0393701 for v in prcp]
df_prcp = df_prcp.set_index('date').resample('M').mean()

In [ ]:
df_temp['avgPrcp']=df_prcp['avgPrcp']
df_temp.head()
df_temp.to_csv("Cali_Weather.csv")

# Washington Avg Temp and Precipitation

In [ ]:
#Create average temperature
dates_temp_W = []
temps_W = []

#for each year from 2015-2019 ...
for year in range(2013, 2017):
    year = str(year)
    print('working on year '+year)
    
    #make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00023129&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are average temperature readings
    avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
    #get the date field from all average temperature readings
    dates_temp_W += [item['date'] for item in avg_temps]
    #get the actual average temperature from all average temperature readings
    temps_W += [item['value'] for item in avg_temps]

#initialize dataframe
df_temp_W = pd.DataFrame()

#populate date and average temperature fields (cast string date to datetime
df_temp_W['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_W]
#convert to degrees F
df_temp_W['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps_W]
df_temp_W = df_temp_W.set_index('date').resample('M').mean()

In [ ]:
### initialize lists to store data
dates_prcp_W = []
prcp_W = []

#for each year from 2015-2019 ...
for year in range(1992, 2017):
    year = str(year)
#     print('working on year '+year)
    
    #make the api call
    r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&stationid=GHCND:USC00046074&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are Precipitation temperature readings
    avg_prcp = [item for item in d['results'] if item['datatype']=='PRCP']
    #get the date field from all average Precipitation readings
    dates_prcp_W += [item['date'] for item in avg_prcp]
    #get the actual average Precipitation from all average Precipitation readings
    prcp_W += [item['value'] for item in avg_prcp]

#initialize dataframe
df_prcp_W = pd.DataFrame()

#populate date and average Precipitation fields (cast string date to datetime
df_prcp_W['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp_W]
#convert to mm to inches
df_prcp_W['avgPrcp'] = [float(v)*0.0393701 for v in prcp_W]
df_prcp_W = df_prcp_W.set_index('date').resample('M').mean()

In [ ]:
df_temp_W['avgPrcp']=df_prcp_W['avgPrcp']
df_temp_W.to_csv("Washington_Weather.csv")

#  Oregon Avg Temp and Precipitation

In [ ]:
#Create average temperature
dates_temp_O = []
temps_O = []

#for each year from 2015-2019 ...
for year in range(2013, 2017):
    year = str(year)
    print('working on year '+year)
    
    #make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00023129&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are average temperature readings
    avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
    #get the date field from all average temperature readings
    dates_temp_O += [item['date'] for item in avg_temps]
    #get the actual average temperature from all average temperature readings
    temps_O += [item['value'] for item in avg_temps]

#initialize dataframe
df_temp_O = pd.DataFrame()

#populate date and average temperature fields (cast string date to datetime
df_temp_O['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_O]
#convert to degrees F
df_temp_O['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps_O]
df_temp_O = df_temp_O.set_index('date').resample('M').mean()

In [ ]:
### initialize lists to store data
dates_prcp_O = []
prcp_O = []

#for each year from 2015-2019 ...
for year in range(1992, 2017):
    year = str(year)
#     print('working on year '+year)
    
    #make the api call
    r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&stationid=GHCND:USC00046074&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are Precipitation temperature readings
    avg_prcp = [item for item in d['results'] if item['datatype']=='PRCP']
    #get the date field from all average Precipitation readings
    dates_prcp_O += [item['date'] for item in avg_prcp]
    #get the actual average Precipitation from all average Precipitation readings
    prcp_O += [item['value'] for item in avg_prcp]

#initialize dataframe
df_prcp_O = pd.DataFrame()

#populate date and average Precipitation fields (cast string date to datetime
df_prcp_O['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp_O]
#convert to mm to inches
df_prcp_O['avgPrcp'] = [float(v)*0.0393701 for v in prcp_O]
df_prcp_O = df_prcp_O.set_index('date').resample('M').mean()

In [ ]:
df_temp_O['avgPrcp']=df_prcp_O['avgPrcp']
df_temp_O.to_csv("Oregon_Weather.csv")

# Add Lat/Long/Elevation

In [ ]:
# create new columns for Lat/Long/Elevation

#California_GHCND:
Nappa_Valley_g='GHCND:USC00046074'
# Washington_GHCND:
Walla_Walla_Valley_g='GHCND:GHCND:USC00457015'
# Oregon_GHCND:
Willamette_Valley_g='GHCND:USW00024232'
#--------------------------------------------------------Latitude-----------------------------------------------------#

#Latitude
#California_LAT:
Nappa_Valley_lat='38.2777'
# Oregon_lat:
Willamette_Valley_lat='44.905'
# Washington_Lat:
Walla_Walla_Valley_lat='46.3119'


#Latitude Red
latitude=[]
for lat in red_USA_df['regions']:
#California
    if lat =='California':
        latitude.append('38.2777')
#Oregon
    elif lat == 'Oregon':
        latitude.append('44.905')
#Washington
    else:
        latitude.append('46.3119')
red_USA_df['lat']=latitude

#Latitude White
latitude=[]
for lat in white_USA_df['regions']:
#California
    if lat =='California':
        latitude.append('38.2777')
#Oregon
    elif lat == 'Oregon':
        latitude.append('44.905')
#Washington
    else:
        latitude.append('46.3119')
white_USA_df['lat']=latitude

#------------------------------------------------------Longitude-------------------------------------------------------------#

#Longitude
#California_Long:
Nappa_Valley_long='-122.2647'
# Oregon_Long:
Willamette_Valley_long='-123.0011'
# Washington_Long:
Walla_Walla_Valley_long= '-119.2633'

#Longitude Red
longitude=[]
for long in red_USA_df['regions']:
#California
    if long =='California':
        longitude.append('-122.2647')
#Oregon
    elif long == 'Oregon':
        longitude.append('-123.0011')
#Washington
    else:
        longitude.append('-119.2633')
red_USA_df['long']=longitude 

#Longitude White
longitude=[]
for long in white_USA_df['regions']:
#California
    if long =='California':
        longitude.append('-122.2647')
#Oregon
    elif long == 'Oregon':
        longitude.append('-123.0011')
#Washington
    else:
        longitude.append('-119.2633')
white_USA_df['long']=longitude 

#-----------------------------------------------------------------Elevation------------------------------------------------#

#Elevation (feet)
#California_Elevation:
Nappa_Valley_long='35.1'
# Oregon_Elevation:
Willamette_Valley_long='205.1'
# Washington_Elevation:
Walla_Walla_Valley_long= '631.9'


#Elevation Red
elevation=[]
for ele in red_USA_df['regions']:
#California
    if ele =='California':
        elevation.append('35.1')
#Oregon
    elif ele == 'Oregon':
        elevation.append('205.1')
#Washington
    else:
        elevation.append('631.9')
red_USA_df['elevation']=elevation

#Elevation White
elevation=[]
for ele in white_USA_df['regions']:
#California
    if ele =='California':
        elevation.append('35.1')
#Oregon
    elif ele == 'Oregon':
        elevation.append('205.1')
#Washington
    else:
        elevation.append('631.9')
white_USA_df['elevation']=elevation


In [ ]:
white_USA_df.head(2)

# Add Temp/Precipitation

In [ ]:
# Split the date into month and year columns
#-------------------------------------------------------------------------------California----------------------------
df_temp.reset_index(inplace= True )
df_temp['day'] = df_temp['date'].dt.day
df_temp['month'] = df_temp['date'].dt.month
df_temp['year'] = df_temp['date'].dt.year

list_months=[]
for x in df_temp['month']:
    list_months.append(calendar.month_name[x])
df_temp['months']=list_months
df_temp.drop(['date', 'day','month'], axis=1, inplace=True)
#----------------------------------------------------------------------------------Oregon-------------------------------
df_temp_O.reset_index(inplace= True )
df_temp_O['day'] = df_temp_O['date'].dt.day
df_temp_O['month'] = df_temp_O['date'].dt.month
df_temp_O['year'] = df_temp_O['date'].dt.year

list_months=[]
for x in df_temp_O['month']:
    list_months.append(calendar.month_name[x])
df_temp_O['months']=list_months
df_temp_O.drop(['date', 'day','month'], axis=1, inplace=True)
#-----------------------------------------------------------------------------------Washington---------------------
df_temp_W.reset_index(inplace= True )
df_temp_W['day'] = df_temp_W['date'].dt.day
df_temp_W['month'] = df_temp_W['date'].dt.month
df_temp_W['year'] = df_temp_W['date'].dt.year

list_months=[]
for x in df_temp_W['month']:
    list_months.append(calendar.month_name[x])
df_temp_W['months']=list_months
df_temp_W.drop(['date', 'day','month'], axis=1, inplace=True)

In [ ]:
df_temp.tail(2)

In [ ]:
white_USA_df.head(2)

In [ ]:
list_temp_march=[]
for u,v,x,y,z in zip(white_USA_df['regions'],white_USA_df['vintage'],df_temp['year'], df_temp['avgTemp'], df_temp['months']):
    if v==x and v==2016 and u=='California'and z=='March':
        list_temp_march.append(y)
    elif y==z and y==2016 and x=='Oregon'and v=='March':
        list_temp_march.append(u)
    else:
        list_temp_march.append(u)
white_USA_df['march']=list_temp_march
white_USA_df
        
        
        

In [ ]:
# sorted_white_USA_df=white_USA_df.sort_values(by=['regions'], ascending=False)
# sorted_white_USA_df.head(50)